# FERNANDO LEON FRANCO

In [1]:
import numpy as np
import polars as pl
from bs4 import BeautifulSoup
import os
from colorstreak import Logger
import re
from dataclasses import dataclass


In [2]:
def print_bar(i, cantidad_registros, contexto="PROGRESO"):
    porcentaje = (i + 1) / cantidad_registros * 100
    # Con emojis
    barra = int(50 * (i + 1) / cantidad_registros) * "🟩"
    espacio = int(50 - len(barra)) * "⬛️"

    print(f"\r{contexto}: |{barra}{espacio}| {porcentaje:6.2f}%", end="", flush=True)

# MEJORAR LA CARGA DESDE UN CSV

In [3]:
def generar_csv(path_lectura, path_guardado):
    # Obtener la ruta absoluta del directorio actual
    # Leerlo y convertirlo en un DataFrame .csv para manipularlo mejor
    with open(path_lectura, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
    data = [line.strip().split(":::") for line in lines]


    df = pl.DataFrame(data, schema=["id", "genero", "pais"], orient="row")


    # Guardar el DataFrame como un archivo .csv
    df.write_csv(path_guardado)
    return True

# ==================== Configuración de rutas =====================


base_path = os.getcwd()

# ===================== Creación de CSV de datos de prueba =====================
ruta_lectura_prueba = os.path.join(base_path, "data/author_profiling/es_test/truth.txt")
ruta_guardado_prueba = os.path.join(base_path, "data/author_profiling/es_test/truth_test.csv")

creado = generar_csv(ruta_lectura_prueba, ruta_guardado_prueba)

if creado:
    Logger.info(f'Archivo CSV creado en: "{ruta_guardado_prueba}"')
# ==============================================================================


# ===================== Creación de CSV de datos de entrenamiento ==============
ruta_lectura_entrenamiento = os.path.join(base_path, "data/author_profiling/es_train/truth.txt")
ruta_guardado_entrenamiento = os.path.join(base_path, "data/author_profiling/es_train/truth_train.csv")

creado_entrenamiento = generar_csv(ruta_lectura_entrenamiento, ruta_guardado_entrenamiento)
if creado_entrenamiento:
    Logger.info(f'Archivo CSV creado en: "{ruta_guardado_entrenamiento}"')
# ==============================================================================



[INFO] Archivo CSV creado en: "/Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_test/truth_test.csv"
[INFO] Archivo CSV creado en: "/Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_train/truth_train.csv"


# CARGAR LOS INDICES DESDE TRUTH DESDE EL CSV

In [4]:
def cargar_xml(id_archivo, train=True) -> str:
    ruta_base = os.path.join(base_path, "data/author_profiling/es_test" if not train else "data/author_profiling/es_train")
    ruta_archivo = os.path.join(ruta_base, f"{id_archivo}.xml")
    # Logger.info(f"ruta archivo: {ruta_archivo}")
    
    with open(ruta_archivo, 'r', encoding='utf-8') as file:
        xml_text = file.read()

    return xml_text

# ===================== Cargar CSV de datos  ==================
 
df_indices = pl.read_csv(ruta_guardado_entrenamiento)
print(df_indices.head())

cantidad_registros = len(df_indices)

registros_crudos = [("id_user","xml_doc","pais","genero") for i in range(cantidad_registros)]


for i, reg in enumerate(registros_crudos):
    id_archivo = df_indices['id'][i]
    id_user = id_archivo
    pais = df_indices['pais'][i]
    genero = df_indices['genero'][i]

    xml_text: str = cargar_xml(id_archivo)
    registros_crudos[i] = (id_user, xml_text, pais, genero)

    print_bar(i, cantidad_registros, contexto="CARGA XML")

print()
Logger.info("Carga de archivos XML completada.\n")
id_user, xml_text, pais, genero = registros_crudos[0]


Logger.debug(f"Total de tweets crudos cargados: {len(registros_crudos)} mostrando 1 registro:\n id:{id_user}\n pais:{pais}\n genero:{genero}\n xml_text:\n{xml_text[:50]}...\n")

shape: (5, 3)
┌─────────────────────────────────┬────────┬──────────┐
│ id                              ┆ genero ┆ pais     │
│ ---                             ┆ ---    ┆ ---      │
│ str                             ┆ str    ┆ str      │
╞═════════════════════════════════╪════════╪══════════╡
│ 74bcc9b0882c8440716ff370494aea… ┆ female ┆ colombia │
│ 4639c055f34ca1f944d0137a5aeb79… ┆ female ┆ colombia │
│ 92ffa98bade702b86417b118e8aca3… ┆ female ┆ colombia │
│ 4560c6567afcccef265f048ed117d0… ┆ female ┆ colombia │
│ 393866dfaa80d414c9896cf8723932… ┆ female ┆ colombia │
└─────────────────────────────────┴────────┴──────────┘
CARGA XML: |🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩| 100.00%
[INFO] Carga de archivos XML completada.

[DEBUG] Total de tweets crudos cargados: 4200 mostrando 1 registro:
 id:74bcc9b0882c8440716ff370494aea09
 pais:colombia
 genero:female
 xml_text:
<author lang="es">
	<documents>
		<document><![CDA...



# PROCESAMOS LOS TEXTOS XML Y LOS DEJAMOS EN UN DF

In [5]:
diccionario_idioma = {
    'es': 'Español', 
    'en': 'Inglés', 
    'fr': 'Francés', 
    'de': 'Alemán', 
    'it': 'Italiano', 
    'nl': 'Neerlandés'
}

def limpiar_xml(texto_xml:str):
    soup = BeautifulSoup(texto_xml, 'lxml-xml')   
    lang = str(soup.author.get('lang'))
    idioma = diccionario_idioma[lang] if lang in diccionario_idioma else lang
    
    documentos = soup.find_all('document') # Obtenemos todsa las etiquetas <document>
    tweets = [doc.get_text(separator=" ", strip=True) for doc in documentos] # Extraemos el texto de cada documento
    
    return tweets, idioma



# ===================== Procesamiento de limpieza del XML =====================
cantidad_registros = len(registros_crudos)

registros_procesados = []

for i, (id_user, doc_crudo, pais, genero) in enumerate(registros_crudos):
    lista_tweets_por_usuario, idioma = limpiar_xml(doc_crudo)
    
    for tweet in lista_tweets_por_usuario:
        registros_procesados.append({
            "id_user": id_user,
            "tweet_crudo": tweet,
            "pais": pais,
            "genero": genero,
            "idioma": idioma
        })
    print_bar(i, cantidad_registros, contexto="Progreso registros limpiados")

print()

# ===================== Creamos dataset persistente =====================

df_registros = pl.DataFrame(registros_procesados)
df_registros.write_parquet("data/author_profiling/registros_procesados.parquet")
Logger.info("Guardando archivo")

lf_registros = df_registros.lazy()

# ===================== Muestra =====================


muestra = lf_registros.limit(5).collect()
Logger.debug(f"Total de tweets procesados: {len(registros_procesados)} mostrando 5 primeros registros:\n {muestra}")


Progreso registros limpiados: |🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩| 100.00%
[INFO] Guardando archivo
[DEBUG] Total de tweets procesados: 419998 mostrando 5 primeros registros:
 shape: (5, 5)
┌─────────────────────────────────┬─────────────────────────────────┬──────────┬────────┬─────────┐
│ id_user                         ┆ tweet_crudo                     ┆ pais     ┆ genero ┆ idioma  │
│ ---                             ┆ ---                             ┆ ---      ┆ ---    ┆ ---     │
│ str                             ┆ str                             ┆ str      ┆ str    ┆ str     │
╞═════════════════════════════════╪═════════════════════════════════╪══════════╪════════╪═════════╡
│ 74bcc9b0882c8440716ff370494aea… ┆ Tiene que valer la pena que es… ┆ colombia ┆ female ┆ Español │
│ 74bcc9b0882c8440716ff370494aea… ┆ Tintas chinas, si ven ésto, es… ┆ colombia ┆ female ┆ Español │
│ 74bcc9b0882c8440716ff370494aea… ┆ "Maestro no le abrió!" -Ay qué… ┆ colombia ┆ female ┆ Espa


# Capa de dataclass de utilidd para normalización y limpieza de tweets

In [6]:
from nltk.tokenize import TweetTokenizer
import unicodedata
# ===================== Configuración de Limpieza de tweets =====================


@dataclass(frozen=True)
class ConfigLimpieza:
    normalizar_unicode: bool = True
    a_minusculas: bool = True
    quitar_urls: bool = True
    quitar_menciones: bool = True
    quitar_hashtags: bool = False 


    

@dataclass
class Tweet:
    URL = re.compile(r'https?://\S+', re.I)
    MENCION = re.compile(r'@\w+')
    HASHTAG = re.compile(r'#\w+')
    ESPACIO = re.compile(r'\s+')
    TT = TweetTokenizer()

    @staticmethod
    def limpiar(texto: str) -> str:
        cfg = ConfigLimpieza()
        if cfg.normalizar_unicode:
            texto = unicodedata.normalize("NFKC", texto)
        if cfg.a_minusculas:
            texto = texto.lower()
        if cfg.quitar_urls:
            texto = Tweet.URL.sub(" ", texto)
        if cfg.quitar_menciones:
            texto = Tweet.MENCION.sub(" ", texto)
        if cfg.quitar_hashtags:
            texto = Tweet.HASHTAG.sub(" ", texto)

        texto_limpio = Tweet.ESPACIO.sub(" ", texto).strip()
        return texto_limpio

    @staticmethod
    def tokenizar(texto: str) -> list[str]:
        return [t for t in Tweet.TT.tokenize(texto)]
    




# PIPELINE DE LIMPIEZA Y/O TOKENIZACIÓN DE TWEETS

In [7]:
# ===================== TRANSFORMACIONES EN PIPELINE =====================
"""
Recordemos que los pipelines de polars si es un lazyframe no se ejecutan hasta que se les pida con .collect() (Docoumentación)
"""


lf_registros_crudos = lf_registros


Logger.info("PIPELINE: LIMPIEZA/ TOKENIZACIÓN")
lf_limpio = (
    lf_registros_crudos.with_columns(
        pl.col("tweet_crudo")
          .map_elements(Tweet.limpiar)
          .alias("texto_limpio")
    )
)

Logger.debug(f"Mostrando registros del lazyframe limpio:\n {lf_limpio.limit(2).collect()}")

primer_tweet_limpio = (
    lf_limpio
    .select('texto_limpio')
    .limit(1)
    .collect()
    .to_series()
    .to_list()[0]
)

Logger.info(f"Primer tweet limpio: {primer_tweet_limpio}")
        
lf_tokens = (
    lf_limpio.with_columns(
        pl.col("texto_limpio")
          .map_elements(
              Tweet.tokenizar, 
              return_dtype=pl.List(pl.Utf8), 
              skip_nulls=True
            )
          .alias("tweets_tokenizados")
    )
)

Logger.debug(f"Mostrando registros del lazyframe tokenizado:\n {lf_tokens.limit(2).collect()}")


# ======================== Metodo para extraer el primer tweet tokenizado con polars =====================
primer_tweet_tokenizado = (
    lf_tokens
    .select("tweets_tokenizados") # Seleccionamos la columna que nos interesa
    .limit(1) 
    .collect()
    .item()  # Sirve para extraer el valor cuando el DataFrame es 1x1
)
# ========================================================================================================

Logger.info(f"Primer tweet tokenizado: {primer_tweet_tokenizado}")

for token in primer_tweet_tokenizado:
    Logger.info(f"Token: {token}")
    


[INFO] PIPELINE: LIMPIEZA/ TOKENIZACIÓN
[DEBUG] Mostrando registros del lazyframe limpio:
 shape: (2, 6)
┌──────────────────────┬──────────────────────┬──────────┬────────┬─────────┬──────────────────────┐
│ id_user              ┆ tweet_crudo          ┆ pais     ┆ genero ┆ idioma  ┆ texto_limpio         │
│ ---                  ┆ ---                  ┆ ---      ┆ ---    ┆ ---     ┆ ---                  │
│ str                  ┆ str                  ┆ str      ┆ str    ┆ str     ┆ str                  │
╞══════════════════════╪══════════════════════╪══════════╪════════╪═════════╪══════════════════════╡
│ 74bcc9b0882c8440716f ┆ Tiene que valer la   ┆ colombia ┆ female ┆ Español ┆ tiene que valer la   │
│ f370494aea…          ┆ pena que es…         ┆          ┆        ┆         ┆ pena que es…         │
│ 74bcc9b0882c8440716f ┆ Tintas chinas, si    ┆ colombia ┆ female ┆ Español ┆ tintas chinas, si    │
│ f370494aea…          ┆ ven ésto, es…        ┆          ┆        ┆         ┆ ven ésto,

In [8]:
# ===================== VOCABULARIO GLOBAL =====================

Logger.info("CONSTRUCCIÓN DE VOCABULARIO GLOBAL")

# 1) Aplanamos la lista de tokens a filas
lf_explotado = lf_tokens.explode("tweets_tokenizados")

Logger.debug(f"Mostrando registros del lazyframe explotado:\n {lf_explotado.limit(5).collect()}")

# 2) Conteo de frecuencia global de cada token
lf_conteos = (
    lf_explotado
    .group_by(pl.col("tweets_tokenizados").alias("token"))
    .len()
    .sort("len", descending=True)  # o .sort("token") si quieres orden alfabético

)

Logger.debug(f"Mostrando registros del lazyframe con conteos:\n {lf_conteos.limit(5).collect()}")

# 3) Asignamos un ID de token estable (0..V-1) y persistimos
df_vocab = (
    lf_conteos
    .with_row_index(name="token_id", offset=0)  # reemplaza with_row_count
    .collect()
)

Logger.info(f"Total de tokens únicos en vocabulario: {len(df_vocab)}. Mostrando 5 primeros:\n {df_vocab.head()}")
df_vocab.write_parquet("data/author_profiling/vocabulario.parquet")
# schema: token_id: u32, token: Utf8, len: u32  (len = frecuencia global)

[INFO] CONSTRUCCIÓN DE VOCABULARIO GLOBAL
[DEBUG] Mostrando registros del lazyframe explotado:
 shape: (5, 7)
┌─────────────────┬────────────────┬──────────┬────────┬─────────┬────────────────┬────────────────┐
│ id_user         ┆ tweet_crudo    ┆ pais     ┆ genero ┆ idioma  ┆ texto_limpio   ┆ tweets_tokeniz │
│ ---             ┆ ---            ┆ ---      ┆ ---    ┆ ---     ┆ ---            ┆ ados           │
│ str             ┆ str            ┆ str      ┆ str    ┆ str     ┆ str            ┆ ---            │
│                 ┆                ┆          ┆        ┆         ┆                ┆ str            │
╞═════════════════╪════════════════╪══════════╪════════╪═════════╪════════════════╪════════════════╡
│ 74bcc9b0882c844 ┆ Tiene que      ┆ colombia ┆ female ┆ Español ┆ tiene que      ┆ tiene          │
│ 0716ff370494aea ┆ valer la pena  ┆          ┆        ┆         ┆ valer la pena  ┆                │
│ …               ┆ que es…        ┆          ┆        ┆         ┆ que es…        